In [1]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.preprocessing import RobustScaler
from collections import Counter
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)


from sklearn.decomposition import PCA

from he_svm import preprocess_a_sample, he_svm, preprocess_a_sample_encrypted
import glob

In [2]:
healthy_csvs = ['data/TriaxalBearings/Healthy bearing data/Healthy with pulley.csv']
                # '../data/TriaxalBearings/Healthy bearing data/healthy without pulley.csv']

LEN_SAMPLES = 500

train_samples = []
for f in healthy_csvs:
    df = pd.read_csv(f)
    df = df.iloc[:, 1:]
    dfs = df.groupby(np.arange(len(df))//LEN_SAMPLES)
    [train_samples.append(t[1]) for t in list(dfs)[:-1]]

In [3]:
len(train_samples)

237

In [4]:
len(train_samples[0])

500

# Train a SVM

In [5]:
def preprocess_a_sample(df, windows):
    final_sample = []
    
    for column in df.columns:
        signal = df.loc[:, column]
        
        signal_fft = np.abs(np.fft.rfft(signal))**2
        len_windows = int(len(signal_fft) / windows) - 1
        
        for i in range(windows):
            if i == windows-1:
                final_sample.append(np.mean(signal_fft[i*len_windows:]))
            else:
                final_sample.append(np.mean(signal_fft[i*len_windows:(i+1)*len_windows]))
                
    return np.array(final_sample)

In [6]:
windows = 8

In [7]:
preprocessed_samples_nominal = np.array([preprocess_a_sample(sample, windows) for sample in train_samples])
np.random.seed(42)
np.random.shuffle(preprocessed_samples_nominal)

n = int(len(preprocessed_samples_nominal) * 0.8)
preprocessed_samples_train = preprocessed_samples_nominal[:n]
preprocessed_samples_test = preprocessed_samples_nominal[n:]

svm = OneClassSVM(nu=0.05, kernel='poly', gamma='scale', degree=2)
svm.fit(preprocessed_samples_train)
svm.gamma_value = 1 / ((windows*3) * preprocessed_samples_train.var())  # to put gamma value in svm

# Test
## Test on nominal data

In [8]:
x_predicted = svm.predict(preprocessed_samples_train)
print(f"###############")
print(f"Training samples: {len(x_predicted)}")
print(f"Found nominal: {len([x for x in x_predicted if x == 1])}")
print(f"Found anomalous: {len([x for x in x_predicted if x == -1])}")
print(f"Accuracy nominal training: {len([x for x in x_predicted if x == 1]) / len(x_predicted)}")

x_predicted = svm.predict(preprocessed_samples_test)
print(f"Test samples (nominal): {len(x_predicted)}")
print(f"Found nominal: {len([x for x in x_predicted if x == 1])}")
print(f"Found anomalous: {len([x for x in x_predicted if x == -1])}")
print(f"Accuracy nominal testing: {len([x for x in x_predicted if x == 1]) / len(x_predicted)}")

###############
Training samples: 189
Found nominal: 179
Found anomalous: 10
Accuracy nominal training: 0.9470899470899471
Test samples (nominal): 48
Found nominal: 45
Found anomalous: 3
Accuracy nominal testing: 0.9375


In [9]:
print(f'Max distance: {max(svm.decision_function(preprocessed_samples_train))}')
print(f'Min distance: {min(svm.decision_function(preprocessed_samples_train))}')

Max distance: 1.0878100689276433
Min distance: -0.8507441308006687


# Test on anomalous data

In [10]:
# Define the directory
dir = 'data/TriaxalBearings/'

# Get the list of files in the directory and its subdirectories
files = []
for f in glob.glob(dir + '**/*.csv', recursive=True):
    if 'Healthy' not in f:
        files.append(f)

In [11]:
sides = ['inner', 'outer']
powers = ['100w', '200w', '300w']

multi_index = pd.MultiIndex.from_product([sides, powers], names=['Sides', 'Powers'])
table_accuracy = pd.DataFrame(columns = multi_index) 

for f in sorted(files):
    print(f"File: {f}")
    short_file = f.split("/")[-1][:-4]
    print(short_file)
    
    side = short_file[3:8]
    power = short_file[9:13]
    depth = short_file[:3]
        
    df = pd.read_csv(f)
    df = df.iloc[:, 1:]
    dfs = df.groupby(np.arange(len(df))//LEN_SAMPLES)
    anomalous_samples = [t[1] for t in list(dfs)[:-1]]
    anomaly_samples = np.array([preprocess_a_sample(df, windows) for df in anomalous_samples])

    x_predicted = svm.predict(anomaly_samples)
    
    n_nom = len([x for x in x_predicted if x == 1])
    n_ano = len([x for x in x_predicted if x == -1])
    accuracy = round((n_ano / len(x_predicted)) * 100, 2)
    table_accuracy.loc[depth, (side, power)] = accuracy
    
    print(f"Test samples (anomalous): {len(x_predicted)}")
    print(f"Found nominal: {n_nom}")
    print(f"Found anomalous: {n_ano}")
    print(f"Accuracy: {accuracy}%")
    print(f"###############")

display(table_accuracy)

File: data/TriaxalBearings/0.7mm-bearing-faults/0.7inner-100watt-67V2Iv.csv
0.7inner-100watt-67V2Iv
Test samples (anomalous): 286
Found nominal: 285
Found anomalous: 1
Accuracy: 0.35%
###############
File: data/TriaxalBearings/0.7mm-bearing-faults/0.7inner-200watt-jolm8U.csv
0.7inner-200watt-jolm8U
Test samples (anomalous): 250
Found nominal: 248
Found anomalous: 2
Accuracy: 0.8%
###############
File: data/TriaxalBearings/0.7mm-bearing-faults/0.7inner-300watt-Zo8w7U.csv
0.7inner-300watt-Zo8w7U
Test samples (anomalous): 227
Found nominal: 227
Found anomalous: 0
Accuracy: 0.0%
###############
File: data/TriaxalBearings/0.7mm-bearing-faults/0.7outer-100watt-lB5LIS.csv
0.7outer-100watt-lB5LIS
Test samples (anomalous): 260
Found nominal: 15
Found anomalous: 245
Accuracy: 94.23%
###############
File: data/TriaxalBearings/0.7mm-bearing-faults/0.7outer-200watt-0Pp0qm.csv
0.7outer-200watt-0Pp0qm
Test samples (anomalous): 260
Found nominal: 127
Found anomalous: 133
Accuracy: 51.15%
#############

Sides   inner                outer              
Powers   100w   200w   300w   100w   200w   300w
0.7      0.35    0.8    0.0  94.23  51.15  63.64
0.9     100.0  100.0  100.0  100.0  100.0  100.0
1.1     100.0  100.0  100.0  100.0  99.34  100.0
1.3     100.0  100.0  100.0  100.0  100.0  96.47
1.5     100.0  100.0  100.0  99.62  100.0  100.0
1.7     100.0  100.0  100.0  100.0  100.0  100.0

# Test on encrypted data

In [13]:
from linetimer import CodeTimer
import tenseal as ts
np.set_printoptions(precision=3, suppress=True)

poly_modulus_degree=2**14
coeff_mod_bit_sizes=[60] + [50]*6 + [60]

# Setup TenSEAL context
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_modulus_degree,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes
          )
context.generate_galois_keys()
context.global_scale = 2**50

errors = {}

for f in files:
    error_df = pd.DataFrame(columns=['ID', 'Expected', 'Predicted (enc)', '% Error', 'Correct?', 'Time enc (s)'])
    df = pd.read_csv(f)
    df = df.iloc[:, 1:]
    dfs = df.groupby(np.arange(len(df))//LEN_SAMPLES)
    anomalous_samples = [t[1] for t in list(dfs)[:-1]]
    
    for sample in anomalous_samples[:]:
        anomaly_sample = preprocess_a_sample(sample, windows).reshape(1, -1)
        
        x_expected = svm.decision_function(anomaly_sample)[0]

        enc_time = CodeTimer(silent=True, unit='s')
        with enc_time:
            x_enc_preprocessed = preprocess_a_sample_encrypted(sample, context, windows, None)
            x_enc_predicted = he_svm(x_enc_preprocessed, svm, windows)
            x_predicted = x_enc_predicted[0].decrypt()[0]

        error_df.loc[len(error_df)] = [sample.index.name, x_expected, x_predicted, 
                                       (x_expected-x_predicted)/x_expected, 
                                       np.sign(x_expected) == np.sign(x_predicted),
                                       enc_time.took]

    f = f.split("/")[-1][:-4]
    errors[f] = error_df
    error_df.to_csv(f"results/TriaxalBearings/Errors_{f}.csv")

    print(f"Sensor: {f}")
    print(error_df)

Sensor: 1.7outer-200watt
     ID  Expected  Predicted (enc)       % Error  Correct?  Time enc (s)
0  None -1.814035        -1.814035  3.355487e-08      True      8.473139
Sensor: 1.7outer-100watt
     ID  Expected  Predicted (enc)       % Error  Correct?  Time enc (s)
0  None -1.824547        -1.824547  3.359056e-08      True      8.492496


KeyboardInterrupt: 